![cat](./img-transformer.jpeg)

transformer无先验假设（CNN-局部关联，RNN-有序建模）
因此模型更大，更加端到端
缺点是带来计算复杂度，序列长度平方复杂度
其实可以这样理解深度学习：所有非线性组合构成了最大的模型空间，不同的模型类型比如说CNN、RNN就是加入了一些限制/一些先验假设
使得模型空间缩小，再从中选出学习到训练数据的模型。当然这些限制/先验假设是有道理的，它们基于对数据的认识
比如CNN for images-特征是局部的，与位置无关，RNN for time seqs-历史影响现在
transformer 预测流式 训练并行
训练并行：attention 在计算每一个z_i时，需要的是全体的x_1:N，而并不依赖z_i-1,decoder不使用上一轮预测的输出，而强制使用正确的单词


理解multihead-attention和FNN层
multihead-attention对不同位置单词进行混合，FNN对每个单词的各个特征维度混合，增强表达能力
CNN中多通道卷积对不同位置(height, width)进行混合，单通道1X1卷积对不同通道混合

In [1]:


import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

batch_size = 2

# 单词表大小 1,2,3,4,5,6,7,8代表八个单词
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8 #pytorch中是512

# 序列最大长度
max_src_seq_len = 5
max_tgt_seq_len = 5
max_position_len = 5

#src_len = torch.randint(2, 5,(batch_size,))
#tgt_len = torch.randint(2, 5,(batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)

# 填充，保证不同句子长度一样
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max(src_len)-L)),0) \
           for L in src_len])  # 增加一维才能cat
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max(tgt_len)-L)),0) \
           for L in tgt_len])

#构造embedding，+1是因为有padding-0在 先转成9维的one-hot，再映射到连续的model_dim维度
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)
src_embedding = src_embedding_table(src_seq)
tgt_embedding = tgt_embedding_table(tgt_seq)

pos_mat = torch.arange(max_position_len).reshape((-1,1))
i_mat = torch.pow(10000,torch.arange(0, 8, 2).reshape((1,-1)) / model_dim)

pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:,0::2] = torch.sin(pos_mat / i_mat)
pe_embedding_table[:,1::2] = torch.cos(pos_mat / i_mat)

#看公式，PE其实就是一个矩阵，每行是每个位置的pos向量，维度与model_dim一致
#用三角函数可以对超出最大长度的位置通过线性组合得到编码
pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)),0) for _ in src_len])
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)),0) for _ in tgt_len])
src_pe_embedding = pe_embedding(src_pos)
tgt_pe_embedding = pe_embedding(tgt_pos)
print(src_seq)
print(src_embedding)
print(src_pe_embedding)

tensor([[1, 5, 0, 0],
        [3, 2, 6, 1]])
tensor([[[-4.3997e-01,  8.7886e-01,  4.1313e-01, -3.3246e-01,  9.6338e-01,
          -1.2753e+00,  1.7734e+00,  3.6435e-01],
         [ 1.9675e+00,  1.3144e-01,  1.9894e+00, -5.8384e-01, -1.8421e+00,
          -6.1899e-01,  8.9139e-01, -2.3314e+00],
         [ 6.6242e-01, -8.4870e-01, -3.6098e-01, -5.3415e-02,  2.1413e-03,
           1.9832e+00, -7.6364e-01, -1.4317e+00],
         [ 6.6242e-01, -8.4870e-01, -3.6098e-01, -5.3415e-02,  2.1413e-03,
           1.9832e+00, -7.6364e-01, -1.4317e+00]],

        [[ 8.0420e-02, -6.0464e-02,  4.8834e-01,  6.3332e-01, -3.8809e-02,
           3.4222e-01, -1.5584e+00,  9.9493e-01],
         [-3.6753e-01,  8.5274e-01, -1.4921e+00,  2.7125e-01, -1.7552e+00,
           9.5978e-01, -2.9227e-02,  1.7589e-01],
         [ 6.2683e-01,  9.2709e-01, -1.0865e+00, -6.5069e-01,  1.3153e-01,
          -2.9367e-01,  6.1597e-01,  9.7760e-01],
         [-4.3997e-01,  8.7886e-01,  4.1313e-01, -3.3246e-01,  9.6338e-01,
   

In [2]:
#attention中做归一化的必要性
score = torch.randn(5)
print(F.softmax(score, -1))
print(F.softmax(score*0.1, -1))
print(F.softmax(score*10, -1))
def softmax_func(score):
    return F.softmax(score,-1)
jacob_mat1 = torch.autograd.functional.jacobian(softmax_func, score*0.1)
jacob_mat2 = torch.autograd.functional.jacobian(softmax_func, score*10)
print(jacob_mat1)
print(jacob_mat2)#softmax函数对绝对值大的数，梯度接近0，如果不归一化，参数无法得到更新

tensor([0.2116, 0.1583, 0.1951, 0.3541, 0.0810])
tensor([0.2031, 0.1973, 0.2014, 0.2138, 0.1845])
tensor([5.7437e-03, 3.1515e-04, 2.5493e-03, 9.9139e-01, 3.8748e-07])
tensor([[ 0.1618, -0.0401, -0.0409, -0.0434, -0.0375],
        [-0.0401,  0.1583, -0.0397, -0.0422, -0.0364],
        [-0.0409, -0.0397,  0.1608, -0.0431, -0.0372],
        [-0.0434, -0.0422, -0.0431,  0.1681, -0.0394],
        [-0.0375, -0.0364, -0.0372, -0.0394,  0.1504]])
tensor([[ 5.7107e-03, -1.8102e-06, -1.4642e-05, -5.6943e-03, -2.2256e-09],
        [-1.8102e-06,  3.1506e-04, -8.0341e-07, -3.1244e-04, -1.2212e-10],
        [-1.4642e-05, -8.0341e-07,  2.5428e-03, -2.5273e-03, -9.8778e-10],
        [-5.6943e-03, -3.1244e-04, -2.5273e-03,  8.5344e-03, -3.8414e-07],
        [-2.2256e-09, -1.2212e-10, -9.8778e-10, -3.8414e-07,  3.8748e-07]])


In [3]:
#src_mask
#Encoder mask的shape:[batch_size, max_src_len,max_src_len],值为1或-inf
#softmax(QK^T/根号d_k) · V
#Q[batch_size,max_src_len,model_dim]
#K^T[batch_size,model_dim,max_src_len]
#QK^T/根号d_k [batch_size,max_src_len,max_src_len]

#先得到有效位置
valid_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)), 0) for L in src_len])
print(src_len)
print(valid_encoder_pos)
valid_encoder_pos = torch.unsqueeze(valid_encoder_pos, 2)
print(valid_encoder_pos.shape)
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1, 2))
print(valid_encoder_pos_matrix)
invalid_encoder_pos_matrix = 1 - valid_encoder_pos_matrix
print(invalid_encoder_pos_matrix)
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
print(mask_encoder_self_attention)

tensor([2, 4], dtype=torch.int32)
tensor([[1., 1., 0., 0.],
        [1., 1., 1., 1.]])
torch.Size([2, 4, 1])
tensor([[[1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
tensor([[[0., 0., 1., 1.],
         [0., 0., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])


In [4]:
score = torch.randn(batch_size, max(src_len), max(src_len))
print(score)
mask_score = score.masked_fill(mask_encoder_self_attention, -1e9)
print(src_len)
print(F.softmax(mask_score, -1))

tensor([[[ 0.3471,  0.2101,  0.6273,  0.7173],
         [-0.5677,  0.5013, -0.3402, -0.3344],
         [ 0.4589,  0.9505, -1.0583,  1.7169],
         [ 0.9693,  1.4873,  1.3005,  0.1534]],

        [[-0.9100, -0.2122,  1.5468,  0.4344],
         [ 0.2642,  0.3578,  0.8848, -1.4179],
         [ 0.7050, -1.6503, -0.9525,  0.3299],
         [-2.5783, -0.3949, -1.2094,  0.3069]]])
tensor([2, 4], dtype=torch.int32)
tensor([[[0.5342, 0.4658, 0.0000, 0.0000],
         [0.2556, 0.7444, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500]],

        [[0.0540, 0.1085, 0.6303, 0.2072],
         [0.2413, 0.2650, 0.4488, 0.0449],
         [0.5069, 0.0481, 0.0966, 0.3484],
         [0.0315, 0.2799, 0.1239, 0.5646]]])


In [12]:
# memory_mask
# 构造intra-attention的mask
# Q @ K^T shape: [batch_size,max(tgt_len),max(src_len)]
# Query是tgt，Key和Value都是src
# [max(tgt_len)-,d_model] [d_model,max(src_len)] [max(src_len),d_model]
# d_model都一样，即使src和target单词表大小不一样，也会映射到一样
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)), 0) for L in src_len]), 2)
print(valid_encoder_pos)
valid_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len)-L)), 0) for L in tgt_len]), 2)
print(valid_decoder_pos)
valid_cross_pos_matrix = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1, 2))
print(valid_cross_pos_matrix)
invalid_cross_pos_matrix = 1 - valid_cross_pos_matrix
mask_cross_self_attention = invalid_cross_pos_matrix.to(torch.bool)
print(mask_cross_self_attention)

tensor([[[1.],
         [1.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.]]])
tensor([[[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [0.]]])
tensor([[[1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [0., 0., 0., 0.]]])
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [ True,  True,  True,  True]]])


In [25]:
# decoder self-attention mask  -- Autoregressive
# 训练阶段做预测，不能给答案  因果mask,流式输出  tgt_mask
valid_decoder_tri_matrix = torch.cat([F.pad(torch.tril(torch.ones((L, L))), (0, max(tgt_len)-L, 0,max(tgt_len)-L))\
                                      .unsqueeze(0) \
                                      for L in tgt_len])
print(valid_decoder_tri_matrix)
invalid_decoder_tri_matrix = (1 - valid_decoder_tri_matrix).to(torch.bool)
print(invalid_decoder_tri_matrix)
score = torch.randn(batch_size, max(tgt_len), max(tgt_len))
print(F.softmax(score, dim=-1))
mask_score = score.masked_fill(invalid_decoder_tri_matrix, -1e9)
print(F.softmax(mask_score, dim=-1))

tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [0., 0., 0., 0.]]])
tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]]])
tensor([[[0.2296, 0.6576, 0.0278, 0.0849],
         [0.1790, 0.2575, 0.0588, 0.5047],
         [0.3294, 0.4897, 0.1174, 0.0635],
         [0.0836, 0.1142, 0.1423, 0.6599]],

        [[0.4590, 0.1518, 0.1646, 0.2246],
         [0.0953, 0.0600, 0.1029, 0.7418],
         [0.4250, 0.1359, 0.2774, 0.1616],
         [0.2813, 0.0973, 0.3286, 0.2927]]])
tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.4101, 0.5899, 0.0000, 0.0000],
         [0.3517, 0.5229, 0.12

In [26]:
def scaled_dot_product_attention(Q, K, V, attn_mask):
    #shape of Q,K,V [batch_size*num_head, seq_len, model_dim/num_head] 
    #nbatch不同，num_head相同时，小矩阵是一样的
    #num_head不同时，小矩阵不一样
    #相当于纵向cat起来的多个小矩阵
    score = torch.bmm(Q, K.transpose(-2,-1))/torch.sqrt(model_dim)
    masked_score = score.masked_fill(attn_mask)
    prob = F.softmax(masked_score, -1)
    context = torch.bmm(prob, V)
    return context

-----loss的mask计算-----

In [44]:
#loss
bs,seq_len,vocab_size = 2, 3, 4
logits = torch.randn(bs,seq_len,vocab_size) #bs,seq_len,vocab_size
label = torch.randint(0, vocab_size, (bs, seq_len))
print(logits.transpose(1, 2)) #bs,vocab_size,seq_len
print(logits.transpose(1, 2).shape), print(label)

tensor([[[ 0.0033,  0.5708,  0.1420],
         [-0.6958, -1.1104, -1.4834],
         [ 0.6394, -0.5118,  0.4612],
         [ 0.4809,  0.3321,  0.7831]],

        [[-0.1615, -2.1341,  1.1805],
         [ 0.1152,  0.9004,  1.1450],
         [-0.3078,  0.3988, -0.0766],
         [ 0.0671,  1.3596,  0.4715]]])
torch.Size([2, 4, 3])
tensor([[3, 0, 1],
        [2, 3, 0]])


(None, None)

In [46]:
#CrossEntropyLoss接受输入shape:[minibatch, Classes,d1,d2,...,dK]
F.cross_entropy(logits.transpose(1, 2), label)

tensor(1.4089)

In [50]:
F.cross_entropy(logits.transpose(1, 2), label, reduction='none')

tensor([[1.1315, 0.8384, 3.1231],
        [1.6368, 0.7153, 1.0086]])

In [57]:
#假设实际输出长度是2，3
tgt_len = torch.Tensor([2, 3]).to(torch.int32)
mask = torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len) - L)), 0) for L in tgt_len])
mask

tensor([[1., 1., 0.],
        [1., 1., 1.]])

In [58]:
F.cross_entropy(logits.transpose(1, 2), label, reduction='none') * mask

tensor([[1.1315, 0.8384, 0.0000],
        [1.6368, 0.7153, 1.0086]])

另外，可以使用pytorch提供的API：ignore_index (int, optional) – Specifies a target value that is ignored and does not contribute to the input gradient. When size_average is True, the loss is averaged over non-ignored targets. Note that ignore_index is only applicable when the target contains class indices. Default: -100

In [59]:
label[0, 2] = -100
F.cross_entropy(logits.transpose(1, 2), label, reduction='none')

tensor([[1.1315, 0.8384, 0.0000],
        [1.6368, 0.7153, 1.0086]])